# 03 Développement des modèles de recommandation

Ce notebook implémente deux approches principales :

1. **Collaborative Filtering (CF)** via Alternating Least Squares (ALS) pondéré par BM25.
2. **Content-Based Filtering (CB)** à base de TF-IDF sur les métadonnées des vidéos.

Chaque modèle est encapsulé dans une classe Python, facilitant l'entraînement (`fit`) et la génération de recommandations (`recommend`).

## 1. Imports et définitions de classes

In [2]:
import scipy.sparse as sp
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight
from typing import Any, Dict, List, Optional
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import joblib

/home/enroot/epita/s8/reco_sys/FinalProject_2025_lucas.collemare/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1.1. Classe `CFModel`
Cette classe wrappe un modèle ALS de la librairie `implicit` :
- **Paramètres** : nombre de facteurs latents, régularisation, itérations, poids global (`alpha`), et hyperparamètres BM25 (`K1`, `B`).
- **`fit`** :
  1. Transpose la matrice user–item en item–user.
  2. Applique un poids BM25 pour atténuer l'effet des items très fréquents.
  3. Entraîne l’ALS.
- **`recommend`** :
  1. Traduit l’`user_id` en index via `user_map`.
  2. Extrait les `N` recommandations non vues.
  3. Retourne la liste des `video_id` correspondants.

In [3]:
class CFModel:
    def __init__(self, factors: int = 128, regularization: float = 0.01, iterations: int = 40, alpha: float = 40.0, K1: float = 100, B: float = 0.8):
        # Initialisation des hyperparamètres
        self.factors = factors
        self.reg = regularization
        self.iter = iterations
        self.alpha = alpha
        self.K1 = K1
        self.B = B
        # Modèle ALS de la librairie implicit
        self.model = AlternatingLeastSquares(factors=self.factors, regularization=self.reg, iterations=self.iter)
        self.user_items: Optional[sp.csr_matrix] = None

    def fit(self, interaction_matrix: sp.csr_matrix):
        # Mémorisation de la matrice user–item (csr)
        self.user_items = interaction_matrix.tocsr()
        # Construction de la matrice item–user
        item_user = self.user_items.T
        # Pondération BM25
        weighted = bm25_weight(item_user, K1=self.K1, B=self.B)
        # Entraînement ALS
        self.model.fit(weighted)

    def recommend(self, user_id: Any, user_map: Dict[Any, int], video_map: Dict[Any, int], interaction_matrix: Optional[sp.csr_matrix] = None, N: int = 10) -> List[Any]:
        # Conversion user_id → index
        uidx = user_map.get(user_id)
        if uidx is None:
            return []
        # Choix de la matrice d'interaction (passée ou stockée)
        if interaction_matrix is not None:
            user_items = interaction_matrix.tocsr()
        elif self.user_items is not None:
            user_items = self.user_items
        else:
            return []
        # Recommandation ALS
        ids, scores = self.model.recommend(uidx, user_items, N=N, filter_already_liked_items=True)
        # Inversion du mapping pour retrouver les video_id
        inv_video_map = {v: k for k, v in video_map.items()}
        return [inv_video_map[i] for i in ids]


## 1.2. Classe `ContentModel`
Ce modèle crée un profil TF-IDF pour chaque vidéo puis en déduit un profil utilisateur :
- TF-IDF sur le champ `feat` (liste de tokens) des métadonnées.
- Alignement des vecteurs TF-IDF sur l’ordre du `video_map`.
- Profil utilisateur = produit normalisé matrice d’interaction × TF-IDF.
- **`recommend`** : similarité cosinus entre profil utilisateur et TF-IDF vidéos.

In [4]:
class ContentModel:
    def __init__(self, max_features: int = 10000, ngram_range=(1, 2), stop_words="english"):
        # Hyperparamètres TF-IDF
        self.max_features = max_features
        self.ngram_range = ngram_range
        self.stop_words = stop_words
        self.tfidf = TfidfVectorizer( max_features=self.max_features, ngram_range=self.ngram_range, stop_words=self.stop_words)
        self.video_ids = None
        self.tfidf_matrix = None
        self.user_profiles = None
        self.user_map = None
        self.vid_map = None

    def fit(self, metadata_df: pd.DataFrame, interaction_matrix: sp.csr_matrix, user_map: dict, video_map: dict, text_field: str = "feat"):
        # Conversion des listes en texte brut
        def to_text(x):
            if isinstance(x, (list, tuple, np.ndarray)):
                return " ".join(str(tok) for tok in x)
            if pd.isna(x):
                return ""
            return str(x)
        # Construction du corpus
        corpus = metadata_df[text_field].apply(to_text).tolist()
        tfidf_full = self.tfidf.fit_transform(corpus)
        # Alignement de l'ordre des vidéos
        all_video_ids = metadata_df["video_id"].tolist()
        ordered_videos = [None] * len(video_map)
        for vid, idx in video_map.items():
            ordered_videos[idx] = vid
        id2row = {v: i for i, v in enumerate(all_video_ids)}
        rows = [id2row[vid] for vid in ordered_videos]
        self.tfidf_matrix = tfidf_full[rows, :]
        self.video_ids = ordered_videos
        # Normalisation des interactions par utilisateur
        um = interaction_matrix.astype("float32")
        row_sums = np.array(um.sum(axis=1)).flatten() + 1e-9
        um = um.multiply(1.0 / row_sums[:, None])
        # Profils utilisateurs = interaction × TF-IDF
        self.user_profiles = um.dot(self.tfidf_matrix).toarray()
        
		# Sauvegarde des artefacts
        sparse.save_npz("models/tfidf_matrix.npz", self.tfidf_matrix)
        joblib.dump(self.tfidf, "models/tfidf_vectorizer.pkl")
        joblib.dump(self.user_profiles, "models/user_profiles.npy")
        joblib.dump(user_map, "models/user_map_content.pkl")
        joblib.dump(video_map, "models/video_map_content.pkl")
        self.user_map = user_map
        self.vid_map = video_map
        print("ContentModel: models and profiles saved under models/")

    def recommend(self, user_id, N: int = 10) -> list:
        # Chargement à la volée si nécessaire
        if self.user_profiles is None:
            self.tfidf_matrix = sparse.load_npz("models/tfidf_matrix.npz")
            self.user_profiles = joblib.load("models/user_profiles.npy")
            self.tfidf = joblib.load("models/tfidf_vectorizer.pkl")
            self.user_map = joblib.load("models/user_map_content.pkl")
            self.vid_map = joblib.load("models/video_map_content.pkl")

        inv_vid_map = {v: k for k, v in self.vid_map.items()}
        uidx = self.user_map.get(user_id)
        if uidx is None:
            return []
        profile = self.user_profiles[uidx].reshape(1, -1)
        sims = cosine_similarity(profile, self.tfidf_matrix).flatten()
        best = np.argpartition(-sims, N)[:N]
        best = best[np.argsort(-sims[best])]
        return [inv_vid_map[i] for i in best]


## 2. Entraînement et sauvegarde des modèles
On charge les features générées précédemment et on entraîne successivement CFModel puis ContentModel.

In [5]:
def load_features():
    # Chargement des données et mappings
    mat = sp.load_npz("features/interaction_matrix.npz")
    user_map = joblib.load("features/user_map.pkl")
    video_map = joblib.load("features/video_map.pkl")
    metadata = pd.read_parquet("preprocessed/item_categories.parquet")
    return mat, user_map, video_map, metadata

# Entraînement du Collaborative Filtering (CF)
mat, user_map, video_map, metadata = load_features()
model = CFModel(factors=64, regularization=0.05, iterations=20, alpha=40.0)
model.fit(mat)
joblib.dump(model, f"models/CF_model.pkl")
print(f"CF Model saved to models/")

# Entraînement du Content-Based (CB)
model = ContentModel(3000, (1,2))
model.fit(metadata_df=metadata, interaction_matrix=mat, user_map=user_map, video_map=video_map, text_field="feat")
joblib.dump(model, f"models/Content-Based_model.pkl")
print(f"Content-Based Model saved to models/")

/home/enroot/epita/s8/reco_sys/FinalProject_2025_lucas.collemare/venv/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/home/enroot/epita/s8/reco_sys/FinalProject_2025_lucas.collemare/venv/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.11811494827270508 seconds
  warnings.warn(
100%|██████████| 20/20 [00:14<00:00,  1.39it/s]


CF Model saved to models/
ContentModel: models and profiles saved under models/
Content-Based Model saved to models/


## 3. Résumé

- **CFModel** : ALS + BM25 pour capturer les similarités collaboratives.
- **ContentModel** : profil TF-IDF pour exploiter les similarités de contenu.
- Les deux modèles sont sauvegardés pour l’étape de génération de recommandations.